# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
from pprint import pprint

# Import API key
from api_keys import g_key
gmaps.configure(api_key = g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
output_data_file = "../output_data/weather_df1.csv"
# output_data_file

types_df = pd.read_csv(output_data_file)

weather_data_df1 = types_df.drop(columns=["Unnamed: 0"])
weather_data_df1

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,puerto ayora,0,ES,1644202568,78,36.5939,-6.2330,54.25,5.75
1,merauke,77,ID,1644202505,63,-8.4667,140.3333,86.92,7.47
2,ahipara,97,NZ,1644202569,96,-35.1667,173.1667,75.11,6.17
3,bredasdorp,13,ZA,1644202570,88,-34.5322,20.0403,65.28,5.75
4,hobyo,98,SO,1644202422,72,5.3505,48.5268,75.52,17.36
...,...,...,...,...,...,...,...,...,...
538,srednekolymsk,30,RU,1644202764,96,67.4500,153.6833,-25.64,1.68
539,wawa,0,NG,1644202858,27,9.9022,4.4192,73.85,5.17
540,port augusta,100,CD,1644202643,89,-7.0349,29.7638,69.89,2.35
541,shimoda,97,JP,1644202779,84,34.6667,138.9500,46.76,1.50


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
locations = weather_data_df1[["Lat", "Lng"]]
# locations

humidity = weather_data_df1["Humidity"].astype(float)

# humidity

# figure_layout = {
#     'width' : '1000px',
#     'height':'400px',
#     'border':'1px solid black',
#     'padding':'1px',
#     'margin':'0 auto 0 auto'
# }

# fig = gmaps.figure(layout=figure_layout)

fig = gmaps.figure()

heatmap_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                   dissipating=False, max_intensity=5,
                                   point_radius=1)

fig.add_layer(heatmap_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Narrow down the DataFrame to find your ideal weather condition. For example:
# A max temperature lower than 80 degrees but higher than 70.
# Wind speed less than 10 mph.
# Zero cloudiness.
# Drop any rows that don't contain all three conditions. 
# You want to be sure the weather is ideal

new_types_df = pd.DataFrame(weather_data_df1, columns = ["City", "Max Temp", "Wind Speed", "Cloudiness"])

max_temp = (new_types_df["Max Temp"] <= 80) & (new_types_df["Max Temp"] > 70)
wind_speed = new_types_df["Wind Speed"] < 10
cloudiness = new_types_df["Cloudiness"] == 0

# new_types_df[max_temp & wind_speed & cloudiness]
new_types_df[max_temp & wind_speed & cloudiness]

,City,Max Temp,Wind Speed,Cloudiness
25,cape town,74.62,0.47,0
170,kayes,73.45,5.99,0
183,san patricio,70.20,8.03,0
192,san cristobal,70.20,8.03,0
208,saint-pierre,78.44,4.61,0
212,kyabe,70.77,6.60,0
359,jiwani,70.41,6.17,0
386,san ramon,70.20,8.03,0
387,san quintin,70.20,8.03,0
417,san vicente,70.20,8.03,0


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = pd.DataFrame(weather_data_df1, columns=["City", "Country", "Lat", "Lng"])

hotel_df["Hotel Name"] = ""
# hotel_df["Coordinates"]=""
hotel_df

,City,Country,Lat,Lng,Hotel Name
0,puerto ayora,ES,36.5939,-6.2330,
1,merauke,ID,-8.4667,140.3333,
2,ahipara,NZ,-35.1667,173.1667,
3,bredasdorp,ZA,-34.5322,20.0403,
4,hobyo,SO,5.3505,48.5268,
...,...,...,...,...,...
538,srednekolymsk,RU,67.4500,153.6833,
539,wawa,NG,9.9022,4.4192,
540,port augusta,CD,-7.0349,29.7638,
541,shimoda,JP,34.6667,138.9500,


In [7]:
# coordinates_lat_lng = []

# target_city = hotel_df["City"]
# # target_city

# target_type = "hotel"

# hotel_df["Coordinates"] = hotel_df["Lat"] + "," + hotel_df["Lng"]

# radius = 5000

# params = {
#     "location:" : coordinates_lat_lng,
#     "types" : target_type,
#     "radius" : radius, 
#     "key" : g_key
# }

# base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# print("A HOTEL WITHIN A 5000 METER RADIUS")

# response = requests.get(base_url, params)
# # print(response.url)

# hotel_distance = response.json()
# print(json.dumps(hotel_distance, indent=4, sort_keys=True))

target_coordinates = f"{hotel_df['Lat'][0]},{hotel_df['Lng'][0]}"
target_search = "hotel"
target_radius = 5000
target_type = "hotel"

# set up a parameters dictionary
params = {
    "location": target_coordinates,
    "keyword": target_search,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
}

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# run a request using our params dictionary
response = requests.get(base_url, params=params)

In [8]:
# base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
# params = {
#     "location": (hotel_df["Lat"], hotel_df["Lng"]),
#     "rankby": "distance",
#     "type" : "hotel", 
#     "key" : g_key
# }

# # params
pprint(response.json(), depth=1)

{'html_attributions': [], 'results': [...], 'status': 'OK'}


In [9]:
hotel_df.iloc[0][0]

'puerto ayora'

In [10]:
#creates a list to capture hotel_names to insert into dataframe
hotel_name = []

# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotel",
    "key": g_key
}

# taken from Airpost_Ratings Example done in class
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
    # convert to json
    name_address = name_address.json()
    # print(json.dumps(name_address, indent=4, sort_keys=True))
    try:
#         hotel_df.iloc[index, 4] = name_address['results'][0]['name']
        hotel_name.append(name_address['results'][0]['name'])
    except IndexError:
        hotel_name.append(np.nan)

In [11]:
hotel_name

['Hotel Puertobahia & Spa',
 nan,
 'Ahipara Bay Motel',
 'Victoria Hotel - Bredasdorp',
 'Osmani Hotel',
 nan,
 'Hotel Plaza Lebu',
 nan,
 'Falls Hotel',
 'HOTEL BURKNER APIAI',
 nan,
 'Hotel Bakkin',
 'The Islander Hotel',
 'Gostinitsa "Arkhara"',
 nan,
 'Hotel Keyanike',
 'Misty Waves Boutique Hotel',
 nan,
 'Legend Garden Hotel',
 nan,
 'Thuwunna Bomi Mountain View Hotel & Resort',
 'Hengifoss Guesthouse',
 'Kak Doma',
 'Beachfront Hotel Hokitika',
 "ABWIK Hotel's",
 'Hotel Molino Viejo',
 'La Quinta Inn & Suites by Wyndham Jamestown',
 'Hotel Castelo',
 'Hotel R9 The Yard',
 nan,
 'Central Palace Hotel',
 'Katsuura Hilltop Hotel & Residence',
 'Chukotka hotel',
 'Airotel Achaia Beach',
 'Arakur Ushuaia Resort & Spa',
 'The Sebel Busselton',
 'Café de Upernavik',
 'People ThankYou',
 nan,
 'Graciosa Resort - Biosphere Island Hotel',
 'King Eider Inn',
 'Hôtel Les Embruns Du Baril',
 'Nome Nugget Inn',
 'Acqua Suítes Maceió',
 'Marawi Resort Hotel',
 'Hotel Keyanike',
 nan,
 'Hilo Re

In [16]:
hotel_df['Hotel Name']= hotel_name
hotel_df.dropna()
hotel_df.to_csv('hotel_export1.csv')

In [17]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [18]:
# hotel_info
# locations

In [19]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))